<a href="https://colab.research.google.com/github/INLANA01/capstone_project_coref_with_transformers/blob/main/My_own_SPAN_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [41]:
import re
import pandas as pd
import spacy
from spacy.symbols import PROPN, PERSON
from __future__ import unicode_literals, print_function
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
from collections import defaultdict
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Install Transformwe libraries

In [21]:
!pip install transformers

# Visualization of model which we are going to use

In [22]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModel, AutoTokenizer, AutoModelWithLMHead, ElectraModel, ElectraForMaskedLM

MODEL_PATH = 'bert-large-uncased'

VOCAB = MODEL_PATH

print('== tokenizing ===')
tokenizer = BertTokenizer.from_pretrained(VOCAB)
model = BertForMaskedLM.from_pretrained(MODEL_PATH)
model.eval()

== tokenizing ===


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

# Preprocessing to get words and their types, we need, since we need Nouns and Pronouns

In [42]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

#  This model follows the concepts defined in https://www.aclweb.org/anthology/W19-3811/ and combines this approach with BertForMaskedLM model across all nouns and pronouns phrases for performing full coreference resolution and get coreference clusters on a given text.

In [27]:
names = ["James","John","Robert","Michael","William","David",
"Richard","Joseph","Thomas","Harry","Charles","Christopher","Daniel","Matthew","Anthony",
"Donald","Mark","Paul","Steven","Andrew","Kenneth","Joshua","Kevin","Brian","George",
"Edward","Ronald","Timothy","Jason","Jeffrey","Ryan","Jacob","Gary","Nicholas",
"Eric","Jonathan","Stephen","Larry","Justin","Scott","Brandon","Benjamin","Samuel",
"Frank","Gregory","Raymond","Alexander","Patrick","Jack","Dennis","Jerry","Tyler",
"Aaron","Jose","Henry","Adam","Douglas","Nathan","Peter","Zachary","Kyle","Walter",
"Harold","Jeremy","Ethan","Carl","Keith","Roger","Gerald","Christian","Terry","Sean",
"Arthur","Austin","Noah","Lawrence","Jesse","Joe","Bryan","Billy","Jordan","Albert",
"Dylan","Bruce","Willie","Gabriel","Alan","Juan","Logan","Wayne","Ralph","Roy",
"Eugene", "Randy","Vincent","Russell","Louis","Philip","Bobby","Johnny","Bradley"]

## Find pronoun clusters

In [38]:
from textblob import TextBlob
# text = """Perry has always done a great job of letting us know she feels."""
text ="""Sally studied with famed family therapist Virginia and began to gain tools for reshaping her life."""
# text = """Victoria Chen, CFO of Megabucks Banking, saw her pay jump to $2.3 million, as the 38-year-old became the company’s president. It is widely known that she came to Megabucks from rival Lotsabucks."""
# text = """28-Year-Old Chef Found Dead at San Francisco Mall A 28-year-old chef who had recently moved to San Francisco was found dead in the stairwell of a local mall this week. But the victim's brother says he can't think of anyone who would want to hurt him, saying, "Things were finally going well for him." The body found at the Westfield Mall Wednesday morning was identified as 28-year-old San Francisco resident Frank Galicia, the San Francisco Medical Examiner's Office said. The San Francisco Police Department said the death was ruled a homicide and an investigation is ongoing. The victim's brother, Louis Galicia, told ABC station KGO in San Francisco that Frank, previously a line cook in Boston, had landed his dream job as line chef at San Francisco's Sons & Daughters restaurant six months ago. A spokesperson for Sons & Daughters said they were "shocked and devastated" by his death. "We are a small team that operates like a close knit family and he will be dearly missed," the spokesperson said. Our thoughts and condolences are with Frank's family and friends at this difficult time. Louis Galicia said Frank initially stayed in hostels, but recently, "Things were finally going well for him." "He found an apartment, he was dating a girl," Louis Galicia told KGO. Louis Galicia said he could not think of anyone who would want to hurt his younger brother. He was a kind spirit with a big heart. His way of connecting with the family was always making us a dish, making us dinner," Louis Galicia said. He never wanted to be in any kind of altercation. He was the brother that went with the flow. "With everything else that's going wrong with the world, he was that diamond in the rough that was shining bright every day," he said. Anyone with information is asked to call the SFPD Tip Line at 415-575-4444."""
for spch in """!"#$%&'()*+/:;<=>?@[\]^_`{|}~’""":
    text = text.replace(spch, '')
for spch in """,.""":
    text = text.replace(spch, ' '+spch)
text = text.replace('-', ' ' + '-' + ' ' )
blob = TextBlob(text)
nouns = set(blob.noun_phrases)
hm = defaultdict(list)

text = text.lower()
print(text)
names_to_nouns = dict()
index = 0
for i, noun in enumerate(nouns):
    while names[index].lower() in text:
        index += 1
    text = text.replace(noun, names[index]+' ')
    names_to_nouns[names[index]] = noun
    index += 1
print(names_to_nouns)
text = text.replace('  ',' ')
sents = text.split(' ')
# print(sents)
# print(len(sents))
for i, data in enumerate(preprocess(' '.join(sents))):
    name, typ = data
    # print(i, name, typ)
    if typ == 'PRP' or typ == 'PRP$':
        print(sents[i-1:])
        sents[i] = '[MASK]'
        sentence = ' '.join(sents)
        print(sentence)
        sm = torch.nn.Softmax(dim=0)
        torch.set_grad_enabled(False)

        # set sentence with MASK token, convert to token_ids
        token_ids = tokenizer.encode(sentence, return_tensors='pt')

        # get the position of the masked token
        masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()

        # forward
        output = model(token_ids)
        last_hidden_state = output[0].squeeze(0)
        # only get output for masked token
        # output is the size of the vocabulary
        mask_hidden_state = last_hidden_state[masked_position]
        # convert to probabilities (softmax)
        # giving a probability for each item in the vocabulary
        label = ''
        probability = 0
        probs = sm(mask_hidden_state)
        for noun in names_to_nouns:
            A_id = tokenizer.convert_tokens_to_ids(noun.lower())
            p_A = probs[A_id].item()
            if p_A > probability:
                probability = p_A
                label = noun
        index = 0
        for w in sents[:i+1]:
            # print(w)
            tk = w
            for spch in """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~’""":
                tk = tk.replace(spch, '')
            if tk in names_to_nouns:
                index += len(names_to_nouns[tk].split(' ')) - 1
        print(i, i+index, name)
        hm[label].append((name, i+index))
        sents[i] = name
sen = ' '.join(sents)
for k, v in names_to_nouns.items():
    sen = sen.replace(k, v)
print(sen)
sens = sen.split(' ')
for k, val in hm.items():
    print(names_to_nouns[k], val)
    print(sens[val[-1][-1]])
    # print(sents[val[1]])

sally studied with famed family therapist virginia and began to gain tools for reshaping her life .
{'James': 'sally', 'John': 'virginia', 'Robert': 'famed family therapist'}
['reshaping', 'her', 'life', '.']
James studied with Robert John and began to gain tools for reshaping [MASK] life .
12 14 her
sally studied with famed family therapist virginia and began to gain tools for reshaping her life .
sally [('her', 14)]
her


## Find noun clusters

In [40]:
# print(names_to_nouns)
for i, data in enumerate(preprocess(' '.join(sents))):
    name, typ = data
    # print(i, name, typ)
    if typ =='NNP' and name in names_to_nouns:
        # print(sents[i-1:])
        # print(sents[i], name)
        sents[i] = '[MASK]'
        sentence = ' '.join(sents)
        # print(sentence)
        sm = torch.nn.Softmax(dim=0)
        torch.set_grad_enabled(False)

        # set sentence with MASK token, convert to token_ids
        token_ids = tokenizer.encode(sentence, return_tensors='pt')

        # get the position of the masked token
        masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()

        # forward
        output = model(token_ids)
        last_hidden_state = output[0].squeeze(0)
        # only get output for masked token
        # output is the size of the vocabulary
        mask_hidden_state = last_hidden_state[masked_position]
        # convert to probabilities (softmax)
        # giving a probability for each item in the vocabulary
        label = ''
        probability = 0
        probs = sm(mask_hidden_state)
        for noun in names_to_nouns:
            A_id = tokenizer.convert_tokens_to_ids(noun.lower())
            p_A = probs[A_id].item()
            if p_A >= probability:
                probability = p_A
                label = noun
        index = 0
        for w in sents[:i+1]:
            # print(w)
            tk = w
            for spch in """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~’""":
                tk = tk.replace(spch, '')
            if tk in names_to_nouns:
                index += len(names_to_nouns[tk].split(' ')) - 1
        # print(i, i+index, name)
        hm[label].append((names_to_nouns[name], i+index))
        sents[i] = name
sen = ' '.join(sents)
for k, v in names_to_nouns.items():
    sen = sen.replace(k, v)
# print(sen)
sens = sen.split(' ')
for k, val in hm.items():
    print(names_to_nouns[k], val)
    # print(sens[val[-1][-1]])
    # print(sents[val[1]])

sally [('her', 14), ('sally', 0), ('virginia', 6), ('sally', 0), ('virginia', 6)]
virginia [('famed family therapist', 3), ('famed family therapist', 3)]


# Get GPU information

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12957665200146041571, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15703311680
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7631080507792326548
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]